# Import

In [122]:
import pandas as pd
import ast 
import difflib as dif
from colorama import Fore, Style
import pickle
#for query
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pythainlp.tokenize import word_tokenize
from string import punctuation
import numpy as np
import time

print('OK !')

OK !


# Load Data

### Load code_mapping

In [123]:
code_mapping = pd.read_csv('code_doc_mapping.csv',dtype='str')

In [124]:
code_mapping.head()

,doc_id,name
0,0000,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...
1,0001,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...
2,0002,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...
3,0003,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...
4,0004,หลักเกณฑ์ วิธีการ และเงื่อนไขในการประกอบธุรกิจ...


### Load dict_query

In [125]:
with open('dict_query.pickle', 'rb') as file:
    dict_query = pickle.load(file)

In [126]:
with open('dict_query_list.pickle', 'rb') as file:
    dict_query_list = pickle.load(file)

In [127]:
# Ex code id
code_id = '0000|0001|0002'
print("code_id: ",code_id)
print("เพื่อใช้ในการค้นหา \n")
print("dict_query: ",dict_query[code_id][:100],"...")
print("\nเพื่อใช้ในการแสดงผลต่างและเหมือน \n")
print("dict_query_list: ",dict_query_list[code_id][:20],"...")

code_id:  0000|0001|0002
เพื่อใช้ในการค้นหา 

dict_query:  23 (2563 เรื่อง นำส่งประกาศธนาคารแห่งประเทศไทย เรื่อง การกำหนด หลักเกณฑ์ วิธีการ และเงื่อนไขในการประ ...

เพื่อใช้ในการแสดงผลต่างและเหมือน 

dict_query_list:  ['23', ' ', '(', '2563', ' ', 'เรื่อง', ' ', 'นำ', 'ส่ง', 'ประกาศ', 'ธนาคารแห่งประเทศไทย', ' ', 'เรื่อง', ' ', 'การ', 'กำหนด', ' ', 'หลักเกณฑ์', ' ', 'วิธีการ'] ...


### Load Dict Pair

In [128]:
#dict_pair
with open('dict_pair.pickle', 'rb') as file:
    dict_pair = pickle.load(file)

In [129]:
index_find = 5
print("query_code_id: ",dict_pair['query'][5])
print("mapping_result_id: ",dict_pair['result'][5])
print("Score: ",dict_pair['Score'][5])

query_code_id:  0000|0001|0014
mapping_result_id:  ['0001|0003|0023', '0002|0001|0018', '0002|0003|0023', '0003|0001|0015']
Score:  [0.3864695375956029, 0.7044029412765824, 0.5754118673416391, 0.3265738860112924]


### Load TF-IDF

#### Query Function

In [130]:
def canonicalize(string):
    normalized_tokens = list()
    a = word_tokenize(string, engine = 'newmm')
    for j in a:
        token = j.strip()
        #Add clean statement here 
        if len(token) > 1 and token not in set(punctuation) and token not in ['..','...','ๆๆ']:
            try:
                normalized_tokens.append(token.lower())
            except:
                normalized_tokens.append(token)
                pass
    return normalized_tokens


# A function that given an input query item returns the top-k most similar items 
# by their cosine similarity.
def find_similar(query_vector, td_matrix, top_k = 5):
    cosine_similarities = cosine_similarity(query_vector, td_matrix).flatten()
    related_doc_indices = cosine_similarities.argsort()[::-1]
    return [(index, cosine_similarities[index]) for index in related_doc_indices][0:top_k]

print('OK !')

OK !


In [131]:
tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.sav', 'rb'))
tfidf_term_document_matrix = pickle.load(open('tfidf_term_document_matrix.sav', 'rb'))

/opt/anaconda3/envs/py39thria/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/py39thria/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Step01 query

In [132]:
def change_dict_query_to_df(dict_query):
    index_list = []
    code_id_list = []
    sentence_list = []
    for index,doc in enumerate(list(dict_query.values())):
        code_id_list.append(list(dict_query.keys())[index])
        index_list.append(index)
        sentence_list.append(doc)
    dict_query_df = pd.DataFrame(data={'index':index_list,'code_id':code_id_list,'sentence_list':sentence_list})
    return dict_query_df

In [133]:
dict_query_df = change_dict_query_to_df(dict_query)
dict_query_df.head()

,index,code_id,sentence_list
0,0,0000|0001|0002,23 (2563 เรื่อง นำส่งประกาศธนาคารแห่งประเทศไทย...
1,1,0000|0001|0003,กำหนด หลักเกณฑ์ วิธีการ และเงื่อนไขในการประกอบ...
2,2,0000|0001|0004,ขอนำส่งประกาศธนาคารแห่งประเทศไทย ที่(ฝ่ายงาน) ...
3,3,0000|0001|0006,การประกอบธุรกิจบัตรเครดิต ลง (วันที่)ซึ่งได้ปร...
4,4,0000|0001|0008,พิเศษ 174 ง ลง (วันที่)มาพร้อมนี้ การปรับปรุงป...


In [75]:
def query(query_text,tfidf_term_document_matrix,dict_query):
    transformed = tfidf_vectorizer.transform([query_text])
    query = transformed[0:1]
    np_result = np.array(find_similar(query, tfidf_term_document_matrix, len(list(dict_query.values()))))
    index_list = []
    score_list = []
    df_dict = {'index': index_list, 'score': score_list}

    df_dict['index'] =  np_result[:,0]
    df_dict['score'] =  np_result[:,1]

    df = pd.DataFrame(df_dict)
    df['index'] = df['index'].astype(int)
    df = df.merge(dict_query_df,on='index',how='left')
    df = df[df['score'] >0]
    return df

In [138]:
sentence_query = 'พ.ร.บ.'
print(sentence_query)

พ.ร.บ.


In [140]:
df_query = query(sentence_query,tfidf_term_document_matrix,dict_query)
df_query.head()

,index,score,code_id,sentence_list
0,321,0.190020,0002|0003|0029,) ตามที่ พ.ร.บ. ธุรกิจสถาบันการเงิน พ.ศ. 2551 ...
1,320,0.182846,0002|0003|0028,จากลูกหนี้เหลือร้อยละ 33 ต่อปี (effective rate...
2,161,0.180530,0001|0003|0030,มีทะเบียนรถเป็นประกัน ตามที่ พ.ร.บ. ธุรกิจสถาบ...
3,16,0.179642,0000|0003|0022,ต่อปี ตามที่ พ.ร.บ. ธุรกิจสถาบันการเงิน พ.ศ. 2...
4,7,0.176311,0000|0001|0015,Corporate Governance) รวมถึงเพื่อปรับลดอัตรารว...


In [141]:
# def query(sentence_query):
#     query_code_id_list = list(dict_query.keys())[list(dict_query.values()).index(sentence_query)]
#     return [query_code_id_list] #return multiple code as list

In [142]:
def get_document_info(query_code_id):
    query_code_id_split = query_code_id.split("|")
    doc_id = query_code_id_split[0]
    page_id = query_code_id_split[1]
    sentence_id = query_code_id_split[2]
    doc_name = code_mapping[code_mapping['doc_id'] == doc_id].iloc[0]['name']
    return doc_id, page_id, sentence_id, doc_name

In [143]:
doc_id, page_id, sentence_id, doc_name = get_document_info('0040|0001|0000')

In [144]:
print('Doc_id: ',doc_id)
print('Doc_Name: ',doc_name)
print('page_id: ',page_id)
print('sentence_id: ',sentence_id)

Doc_id:  0040
Doc_Name:  แนวทางการใหค้วามช่วยเหลือลูกหนี้้ีที่ไดร้ับผลกระทบจากโรคติดเช้ือไวรัสโค โรนา 2019 (มาตราการแก้หนี้ระยะยาวด้วยการรีไฟแนนส์(refinance) และการ รวมหนี้้ี(debt consolidation))
page_id:  0001
sentence_id:  0000


# Show Network

In [82]:
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

In [83]:
# doc_id_query = []
# for code_id in df_query['code_id']:
#     doc_id_query.append(code_id.split("|")[0])
#     doc_id_query = list(set(doc_id_query))

In [84]:
def create_network(df_query):
    if df_query.shape[0] != 0:
        doc_id_query = []
        for code_id in df_query['code_id']:
            doc_id_query.append(code_id.split("|")[0])
            doc_id_query = list(set(doc_id_query))
    else:
        doc_id_query = None
    
    data_max_pair = pd.read_csv('data_max_pair4.csv')
    doc_id_A_list = []
    doc_id_B_list = []
    doc_id_list = []
    set_of_pair_list = []
    
    code_id_pairs = list(data_max_pair['max_pair_list'])
    for code_id_pair in code_id_pairs:
        code_id_split = code_id_pair.split(' ~ ')

        doc_id_A = code_id_split[0].split('|')[0]
        doc_id_B = code_id_split[1].split('|')[0]
        
        if doc_id_query != None and doc_id_A in doc_id_query and doc_id_B in doc_id_query:
            doc_id_A_list.append(doc_id_A)
            doc_id_B_list.append(doc_id_B)
            set_of_pair_list.append({doc_id_A,doc_id_B})
            
        elif doc_id_query == None:
            doc_id_A_list.append(doc_id_A)
            doc_id_B_list.append(doc_id_B)
            set_of_pair_list.append({doc_id_A,doc_id_B})
            
    if len(doc_id_A_list) == 0:
        return None

    doc_id_list.extend(doc_id_A_list)
    doc_id_list.extend(doc_id_B_list)
    doc_id_list = list(set(doc_id_list))
    
    G = Network()
    for doc_A in doc_id_list:
        for doc_B in doc_id_list:
            if doc_A != doc_B and {doc_A,doc_B} in set_of_pair_list:
                weight = set_of_pair_list.count({doc_A,doc_B})
                G.add_node(doc_A)
                G.add_node(doc_B)
                # G.add_edge(doc_A, doc_B, weight=weight )
                G.add_edge(doc_A, doc_B, value=weight)
    return G

In [85]:
G = create_network(df_query)
# G.show_buttons(filter_=["nodes", "edges", "physics"])
G.show('nx.html')

In [86]:
# g = Network()
# total = 0
# all_doc = df_all_doc['Doc_ID'].unique()
# for n in range(0,len(all_doc)): 
#     g.add_node(n)
# for i in range(0,len(all_doc)): 
#     for j in range(i,len(all_doc)):
#         if i != j:
#             doc_pair_weigh = doc_pair.count({all_doc[i], all_doc[j]})
#             if doc_pair_weigh > 0:
#                 total += doc_pair_weigh
#                 g.add_edge(i, j, value=doc_pair_weigh)
# g.show_buttons(filter_=['physics'])
# g.show('nx.html')

In [87]:
# def create_network(df_query):
#     if df_query.shape[0] != 0:
#         doc_id_query = []
#         for code_id in df_query['code_id']:
#             doc_id_query.append(code_id.split("|")[0])
#             doc_id_query = list(set(doc_id_query))
#     else:
#         doc_id_query = None
    
#     data_max_pair = pd.read_csv('data_max_pair4.csv')
#     doc_id_A_list = []
#     doc_id_B_list = []
#     doc_id_list = []
#     set_of_pair_list = []
    
#     code_id_pairs = list(data_max_pair['max_pair_list'])
#     for code_id_pair in code_id_pairs:
#         code_id_split = code_id_pair.split(' ~ ')

#         doc_id_A = code_id_split[0].split('|')[0]
#         doc_id_B = code_id_split[1].split('|')[0]
        
#         if doc_id_query != None and doc_id_A in doc_id_query and doc_id_B in doc_id_query:
#             doc_id_A_list.append(doc_id_A)
#             doc_id_B_list.append(doc_id_B)
#             set_of_pair_list.append({doc_id_A,doc_id_B})
            
#         elif doc_id_query == None:
#             doc_id_A_list.append(doc_id_A)
#             doc_id_B_list.append(doc_id_B)
#             set_of_pair_list.append({doc_id_A,doc_id_B})
            
#     if len(doc_id_A_list) == 0:
#         return None

#     doc_id_list.extend(doc_id_A_list)
#     doc_id_list.extend(doc_id_B_list)
#     doc_id_list = list(set(doc_id_list))
    
#     G = nx.Graph()
#     for doc_A in doc_id_list:
#         for doc_B in doc_id_list:
#             if doc_A != doc_B and {doc_A,doc_B} in set_of_pair_list:
#                 weight = set_of_pair_list.count({doc_A,doc_B})
#                 G.add_edge(doc_A, doc_B, weight=weight )
#     return G

In [88]:
# G = create_network(df_query)

In [89]:
# def show_network(pyvis_network):
#     pyvis_network = Network(height = 800,width = 800, notebook=True)
#     pyvis_network.toggle_hide_edges_on_drag(False)
#     pyvis_network.barnes_hut()
#     pyvis_network.show_buttons(filter_=['physics'])
#     return pyvis_network

In [90]:
# pyvis_network = show_network(G)
# pyvis_network.show("result.html")

In [91]:
# nx.draw(G,with_labels=True,font_weight="bold")
# plt.show()

In [92]:
# data_max_pair = pd.read_csv('data_max_pair4.csv')

In [93]:
# data_max_pair.head()

In [94]:
# doc_id_A_list = []
# doc_id_B_list = []
# doc_id_list = []
# set_of_pair_list = []

# code_id_pairs = list(data_max_pair['max_pair_list'])
# for code_id_pair in code_id_pairs:
#     code_id_split = code_id_pair.split(' ~ ')
    
#     doc_id_A = code_id_split[0].split('|')[0]
#     doc_id_B = code_id_split[1].split('|')[0]
# #     if doc_id_A == '0028' and doc_id_B == '0038':
# #         print(code_id_pair)
    
#     doc_id_A_list.append(doc_id_A)
#     doc_id_B_list.append(doc_id_B)
#     set_of_pair_list.append({doc_id_A,doc_id_B})
    
# doc_id_list.extend(code_id_A_list)
# doc_id_list.extend(code_id_B_list)
# doc_id_list = list(set(doc_id_list))

In [95]:
# G = nx.Graph()
# for doc_A in doc_id_list:
#     for doc_B in doc_id_list:
#         if doc_A != doc_B and {doc_A,doc_B} in set_of_pair_list:
#             weight = set_of_pair_list.count({doc_A,doc_B})
#             G.add_edge(doc_A, doc_B, weight=weight )
#             #create edge
            

In [96]:
# nx.draw(G,with_labels=True,font_weight="bold")
# plt.show()

In [97]:
# code_mapping[code_mapping['doc_id'].isin(['0039','0034','0024'])]['name']

# Show result when click on any query
* click on '0001|0003|0023'

In [98]:
def show_result(query_code_id,mapping_result_id):
    query_sentence = dict_query_list[query_code_id]
    compare_sentence = dict_query_list[mapping_result_id]
    
    compare_sentence_result_list = list(dif.Differ().compare(query_sentence,compare_sentence))
    
    new_str1 = ''
    new_str2 = ''
    len_first = 0 #เช็กว่าเป็นคำแรกของประโยคไหม ถ้าเป็นก็จะตัดออก เพื่อปรับให้ประโยคตรงกัน
    
    for symbol_and_word in compare_sentence_result_list:
        symbol = symbol_and_word[:2]
        word = symbol_and_word[2:]
        if symbol == '+ ' and len_first!= 0:
            new_str1 = new_str1 + f"{Fore.RED}{word}"
        elif symbol == '- ' and len_first!= 0:
            new_str2 = new_str2 + f"{Fore.BLUE}{word}"
        elif symbol == '? ':
            None
        elif symbol != '+ ' and symbol != '- ' and symbol != '? ':
            len_first += 1
            new_str1 = new_str1 + f"{Fore.BLACK}{word}"
            new_str2 = new_str2 + f"{Fore.BLACK}{word}"
    
    return new_str2.replace('BLANK',' '),new_str1.replace('BLANK',' ')

In [99]:
def click_query(query_code_id):
    index_query = dict_pair['query'].index(query_code_id)
    index_match_query = dict_pair['result'][index_query]
    index_match_score = dict_pair['Score'][index_query]
    
    result_sentence_list = []
    for result_from_query in index_match_query:
        query_sentence, result_sentence = show_result(query_code_id,result_from_query)
        result_sentence_list.append(result_sentence)
    return query_sentence, result_sentence_list, index_match_query, index_match_score

In [100]:
# index_find = 5
# print("query: ",dict_pair['query'][5])
# print("result: ",dict_pair['result'][5])
# print("Score: ",dict_pair['Score'][5])

In [179]:
query_id = '0002|0024|0001'
query_sentence,result_sentence_list,index_match_query,index_match_score = click_query(query_id)
doc_id, page_id, sentence_id, doc_name = get_document_info(query_id)
print(doc_id,':',page_id)
print(doc_name)

0002 : 0024
นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลักเกณฑ์ วิธีการ และเงื่อนไขในการประกอบธุรกิจสินเชื่อรายย่อยเพื่อการประกอบอาชีพภายใต้การกำกับ


In [191]:
print("Query Result: ")
print(query_sentence)
print('\n')
print("จำนวนผล : ",len(result_sentence_list))
print('\n')
for index , result_sentence in enumerate(result_sentence_list):
    print("Result_Number: ",index)
    doc_id, page_id, sentence_id, doc_name = get_document_info(index_match_query[index])
    print("Doc_id: ",doc_id)
    print("ชื่อเอกสาร: ",doc_name)
    print("หน้าที่:",int(page_id))
    print(result_sentence)
    print("Score: ",index_match_score[index])
    print('\n')
#     break

Query Result: 
) ธุรกิจบัตรเครดิต ธุรกิจสินเชื่อส่วนบุคคลภายใต้การกำกับ ธุรกิจสินเชื่อรายย่อยเพื่อการ ประกอบอาชีพภายใต้การกำกับ ธุรกิจสินเชื่อรายย่อยระดับจังหวัดภายใต้การกำกับ ธุรกิจระบบหรือ เครือข่ายอิเล็กทรอนิกส์สำหรับธุรกรรมสินเชื่อระหว่างบุคคลกับบุคคล ซึ่งเป็นกิจการที่ต้องขออนุญาต ตามกฎหมาย (๓) ธุรกิจระบบการชำระเงินภายใต้การกำกับและธุรกิจบริการการชำระเงินภายใต้การกำกับ ตามกฎหมายว่าด้วยระบบการชำระเงิน (๔) ธุรกิจบริหารสินทรัพย์ตามกฎหมายว่าด้วยบริษัทบริหารสินทรัพย์ (๕) ธุรกิจเกี่ยวกับปัจจัย


จำนวนผล :  5


Result_Number:  0
Doc_id:  0000
ชื่อเอกสาร:  นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลักเกณฑ์ วิธีการ และเงื่อนไขในการประกอบธุรกิจบัตรเครดิต
หน้าที่: 23
) ธุรกิจบัตรเครดิต ธุรกิจสินเชื่อส่วนบุคคลภายใต้การกำกับ ธุรกิจสินเชื่อรายย่อยเพื่อการ ประกอบอาชีพภายใต้การกำกับ ธุรกิจสินเชื่อรายย่อยระดับจังหวัดภายใต้การกำกับ ธุรกิจระบบหรือ เครือข่ายอิเล็กทรอนิกส์สำหรับธุรกรรมสินเชื่อระหว่างบุคคลกับบุคคล ซึ่งเป็นกิจการที่ต้องขออนุญาต ตามกฎหมาย (๓) ธุรกิจระบบการชำระเงินภายใต้การกำกับและธุรกิจ

In [181]:
res_df = {
    'doc_id': [],
    'page_id': [],
    'sentence_id': [],
    'doc_name': [],
    'result_sentence': [],
}
for index , result_sentence in enumerate(result_sentence_list):
    doc_id, page_id, sentence_id, doc_name = get_document_info(index_match_query[index])
    res_df['doc_id'].append(doc_id)
    res_df['page_id'].append(page_id)
    res_df['sentence_id'].append(sentence_id)
    res_df['doc_name'].append(doc_name)
    res_df['result_sentence'].append(result_sentence)
res_df = pd.DataFrame(res_df)

In [177]:
res_df

,doc_id,page_id,sentence_id,doc_name,result_sentence
0,0000,0023,0023,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...,[30m)[30m [30mธุรกิจ[30mบัตรเครดิต[30m [...
1,0001,0029,0006,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...,[30m)[30m [30mธุรกิจ[30mบัตรเครดิต[30m [...
2,0003,0029,0023,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...,[30mการกำกับ[30m [30mธุรกิจ[30mสินเชื่อ[3...
3,0007,0003,0022,กิจการที่ต้องขออนุญาติตามข้อ ๕ แห่งประกาศของคณ...,[30m)[30m [30mธุรกิจ[30mบัตรเครดิต[30m [...
4,0008,0003,0023,กิจการที่ต้องขออนุญาติตามข้อ ๕ แห่งประกาศของคณ...,[30m)[30m [30mธุรกิจ[30mบัตรเครดิต[30m [...


In [178]:
doc_id_list = list(pd.unique(res_df['doc_id'].values))

In [115]:
from ansi2html import Ansi2HTMLConverter
import sys
conv = Ansi2HTMLConverter()
# ansi = "".join(sys.stdin.readlines())
html = conv.convert(result_sentence)

In [116]:
# print(html)

In [108]:
print(query_sentence)

 (1) นโยบายของสถาบันการเงินควรให้ความสําคัญ กับการพิจารณาข้อบ่งชี้ (indicators) ที่จะส่งผล ต่อการลดลงของความสามารถในการชำระหนี้ หรือการเพิ่มขึ้นของความเสี่ยงด้านเครดิตอย่างมี นัยสําคัญ โดยตามแ (ฝ่ายงาน) างสากลควรพิจารณาข้อ บ่งชี้ 2 ด้านสำคัญ ได้แก่ 1) ด้านการค้างชำระ และ 2) ด้านฐานะการเงิน ผลการดำเนินงาน และการชำระหนี้ ซึ่งครอบคลุมทั้งเหตุการณ์ที่ เกิดขึ้นแล้วและอาจเกิดขึ้นในอนาคต (2) หาก


In [165]:
res_df

,doc_id,page_id,sentence_id,doc_name,result_sentence
0,0000,0001,0015,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...,[30m)[30m [31mรวมถึง[30mเพื่อ[31mปรับ[31...
1,0000,0003,0021,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...,[30m [30mตามที่[30m [30mพ.ร.บ.[30m [30mธ...
2,0001,0003,0030,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่องกำหนด หลั...,[30m [30mตามที่[30m [30mพ.ร.บ.[30m [30mธ...


In [166]:
filter_res_df = res_df.copy()

In [168]:
filter_res_df['doc_id'].values[]

SyntaxError: invalid syntax (1264559644.py, line 1)